In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, ConfusionMatrixDisplay
from scipy.stats import randint

In [2]:
df = pd.read_csv('Task 3 and 4_Loan_Data.csv')
df.drop(['customer_id'], axis=1, inplace=True)
df

credit_lines_outstanding  loan_amt_outstanding  total_debt_outstanding  \
0                            0           5221.545193             3915.471226   
1                            5           1958.928726             8228.752520   
2                            0           3363.009259             2027.830850   
3                            0           4766.648001             2501.730397   
4                            1           1345.827718             1768.826187   
...                        ...                   ...                     ...   
9995                         0           3033.647103             2553.733144   
9996                         1           4146.239304             5458.163525   
9997                         2           3088.223727             4813.090925   
9998                         0           3288.901666             1043.099660   
9999                         1           1917.652480             3050.248203   

           income  years_employed  fico_score  default  
0     78039.38546               5         605        0  
1     26648.43525               2         572        1  
2     65866.71246               4         602        0  
3     74356.88347               5         612        0  
4     23448.32631               6         631        0  
...           ...             ...         ...      ...  
9995  42691.62787               5         697        0  
9996  79969.50521               8         615        0  
9997  38192.67591               5         596        0  
9998  50929.37206               2         647        0  
9999  30611.62821               6         757        0  

[10000 rows x 7 columns]

In [3]:
# split data
X_train, X_test = train_test_split(df, test_size=0.2, random_state=42)

# Labels
Y_train = X_train['default']
X_train = X_train.drop('default', axis=1)

Y_test = X_test['default']
X_test = X_test.drop('default', axis=1)

In [4]:
# Find best hyperparameters and train model
param_dist = {'n_estimators': randint(10,1000),
              'max_depth': randint(1,20)}

# Create a random forest classifier
forest = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(forest, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, Y_train)

# Create a variable for the best model
forest = rand_search.best_estimator_
Y_predictions = forest.predict(X_test)

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'max_depth': 15, 'n_estimators': 995}


In [5]:
# test model
accuracy = accuracy_score(Y_test, Y_predictions)
precision = precision_score(Y_test, Y_predictions)
recall = recall_score(Y_test, Y_predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.995
Precision: 0.9941520467836257
Recall: 0.9770114942528736


In [68]:
# function to calculate loss
input = [0.0, 1958.928726, 8228.75252, 26648.43525, 2.0, 572.0]

# Reshape the input to have the correct dimensions
input = pd.DataFrame([input])
input.columns = X_train.columns

# Predict the default probability
default_prob = forest.predict(input)
default_prob = float("".join([str(x) for x in default_prob]))

expected = default_prob if default_prob == 1 else input['loan_amt_outstanding'][0] * 0.1

print(expected)

195.89287260000003
